# 2D Boundary signature experiments
This notebook presents experiments focusing solely on the 2D boundary signature feature for tumor classification. The experiments will explore two primary approaches: single-view and multi-view. In each step, we will utilize either single or multiple slices' features for classification.

For the single-slice approach and the multi-view single-slice approach, we will employ a Support Vector Machine (SVM) classifier. For multi-slice classification, we will utilize SVM and/or Long Short-Term Memory (LSTM) networks as the classifiers.


1. Single view
    1. sagittal view
    2. coronal view
    3. axial view

2. Multi view
    1. sagittal + coronal
    2. sagittal + axial
    3. coronal + axial
    4. sagittal + coronal + axial

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm.notebook import tqdm
import sys

sys.path.append("../")

from abus_classification import utils, datasets, transformers

##  1. Single View
### 1.1 Single Slice

In [13]:
def get_largest_tumor_slice_number(mask):
    
    depth = mask.shape[2]
    ret = 0
    max_area = 0
    for d in range(depth):
        area = mask[:,:,d].sum()
        if area > max_area:
            max_area = area
            ret = d
            
    return ret

#### Normalized signature


$$ X_{std} = (X - X_{min}) / (X_{max} - X_{min}) $$
$$ X_{scaled} = X_{std} * (max - min) + min $$

In [14]:
def normalize(x):
    for i in range(100):
        x[i] = utils.math.normalize(x[i])
    return x

In [15]:
def classify_with_svm(x, y):
    
    acc = 0
    train_acc = 0
    cfm = [[0,0],
           [0,0]]

    for i in range(100):
        x_test, y_test = x[i], y[i]
        X_train, Y_train = x[:i] + x[i+1:], y[:i] + y[i+1:]
        clf = SVC()
        clf.fit(X_train, Y_train)
        P = clf.predict(X_train)
        train_acc += accuracy_score(Y_train, P)
        res = clf.predict([x_test])[0]
        if res == y_test:
            acc += 1
            
        cfm[y_test][res] += 1
            
    print(f"Train accuracy: {train_acc/100}")
    print(f"Accuracy: {acc/100}")
    print(f"{cfm[0]}\n{cfm[1]}")

In [16]:
def calculate_signature_dataset(dataset):
    signature_dataset = []

    for _, sample, label in tqdm(dataset):
        largest_slice_idx = get_largest_tumor_slice_number(sample)
        sample_slice = sample[:,:,largest_slice_idx]
        sample_slice = utils.image.rotation_invariant(sample_slice)
        boundary = utils.image.get_boundary(sample_slice)
        signature = utils.features.boundary_signature_2d(boundary_image=boundary, resolution=1)
        signature_dataset.append((signature, label))
    
    X, Y = zip(*signature_dataset)
    X = list(X)
    Y = list(Y)
    
    return X, Y 

In [17]:
x_axis = 2
y_axis = 1
z_axis = 0

coronal_transformer = transformers.TransposeTransformer(shape=(x_axis, z_axis, y_axis))
axial_transformer = transformers.TransposeTransformer(shape=(x_axis, y_axis, z_axis))

dataset_list = {
                "sagittal": datasets.TDSCTumors("../data/tdsc"),
                "coronal":  datasets.TDSCTumors("../data/tdsc", transforms=[coronal_transformer]),
                "axial":    datasets.TDSCTumors("../data/tdsc", transforms=[axial_transformer])
            } 

In [19]:
for name, dataset in dataset_list.items():
    print(f"Processing {name} view...")
    X, Y = calculate_signature_dataset(dataset)
    print("Not-Normalized:")
    classify_with_svm(X,Y)
    print("Normalized:")
    X = normalize(X)
    classify_with_svm(X,Y)

Processing sagittal view...


  0%|          | 0/100 [00:00<?, ?it/s]

Not-Normalized:
Train accuracy: 0.7294949494949499
Accuracy: 0.61
[39, 19]
[20, 22]
Normalized:
Train accuracy: 0.7501010101010107
Accuracy: 0.66
[43, 15]
[19, 23]
Processing coronal view...


  0%|          | 0/100 [00:00<?, ?it/s]

Not-Normalized:
Train accuracy: 0.6852525252525247
Accuracy: 0.61
[41, 17]
[22, 20]
Normalized:
Train accuracy: 0.6708080808080801
Accuracy: 0.61
[42, 16]
[23, 19]
Processing axial view...


  0%|          | 0/100 [00:00<?, ?it/s]

Not-Normalized:
Train accuracy: 0.7359595959595966
Accuracy: 0.68
[42, 16]
[16, 26]
Normalized:
Train accuracy: 0.7649494949494944
Accuracy: 0.67
[45, 13]
[20, 22]
